In [ ]:
from requests import get, post
from dotenv import load_dotenv, find_dotenv
import os
import pandas as pd
load_dotenv(find_dotenv())

In [ ]:
auth = f"Bearer BQAvliT5WUSg9VYV6HPaDh2RA9PtkCZqzU249YCbipIt_i_8zbIPPGy8PDdaaKBTM70839oPfpxzvu4fMSpAzsbHshI46gD4DmKsLawft5yFyvwY_y_oyggzE22QZ7Ek4YlyHEFsZ5OJzT74_bfEcTV6I04-udQ"
headers = {"Accept": "application/json", "Content-Type": "application/json", "Authorization": auth}


In [ ]:
artists = {}
songs = []

def get_total_song_count():
    playlist_res = get(f'https://api.spotify.com/v1/playlists/{os.getenv("PLAYLIST_ID")}', headers=headers)
    return playlist_res.json()["tracks"]["total"]


def add_genres(list, genres):
    for genre in genres:
        if genre not in list:
            list.append(genre)
    
    return list

def get_song_genres(song):
    genres = []
    for artist in song["track"]["artists"]:
        id = artist["id"]
        if id not in artists:
            artist_genres_res = get(f'https://api.spotify.com/v1/artists/{id}', headers=headers)
            artist_genres = artist_genres_res.json()["genres"]
            genres = add_genres(genres, artist_genres)
            artists[id] = artist_genres
        else:
            genres = add_genres(genres, artists[id])

    return genres

def song_data(song):
    return song["track"]['id'], song["track"]['name']


In [ ]:


all_genres = []
for offset in range(0, get_total_song_count(), 100):
    all_genres_res = get(f'https://api.spotify.com/v1/playlists/{os.getenv("PLAYLIST_ID")}/tracks?limit=100&offset={offset}', headers=headers)
    for song in all_genres_res.json()["items"]:
        (id, name), song_genres = song_data(song), get_song_genres(song)
        songs.append({"id": id, "name": name, "genres": song_genres})
        all_genres = add_genres(all_genres, song_genres)


In [ ]:
playlist = pd.DataFrame(data=songs, columns=['id', 'name', 'genres'])

In [ ]:
playlist['genres'].value_counts()

In [ ]:
import numpy as np
playlist["genres_indexed"] = [[np.any(genre) == all_genres_x for all_genres_x in all_genres]  for genre in playlist['genres']]


In [ ]:
[[[i or j or x for j in playlist['genres_indexed'][0][0]] for i in playlist['genres_indexed'][1][0]] for x in playlist['genres_indexed'][2][0]]

In [ ]:
pd.Series(playlist["genres_indexed"][0]).value_counts()